In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
pd.set_option("display.max_colwidth", 510)
pd.set_option("display.max_columns",500)

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV

In [4]:
from sklearn.metrics import roc_auc_score, roc_curve
from nltk.stem.snowball import EnglishStemmer

_______________________________

# Cocktail recommender:  
User inputs a description of what kind of cocktail she/he likes and the algorithme returns suggestions based on KNN.

## Import data

Clean data, create a working dataframe and a retriever dataframe with all the cocktail information.

In [5]:
df_ctl_info = pd.read_csv('./CSVs/cocktail_info.csv', sep=',')

In [6]:
df_ctl_info.head()

,Name,Description,Recipe,Ingredients,Nutrition Facts,Video Link
0,American Beauty Cocktail,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Woops... We couldn't retrieve the exact recipe... It's trial & error time! Just a little more fun before enjoying a nice drink!,| 1 oz: Brandy | .5 oz: Dry Vermouth | .25 oz: White Creme De Menthe | 1 oz: Orange Juice | 1 tsp: Grenadine Syrup | 1 oz: Red Port,Calories: 185kcal | Carbohydrates: 14g | Protein: 0.3g | Fat: 0.1g | Saturated Fat: 0.01g | Polyunsaturated Fat: 0.02g | Monounsaturated Fat: 0.01g | Sodium: 6mg | Potassium: 89mg | Fiber: 0.1g | Sugar: 10g | Vitamin A: 50IU | Vitamin C: 20.6mg,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
1,Azzuro,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Shake and strain into an ice-filled collins glass. and garnish with fruit.,| 1 oz: Exotic-Fruit Liqueur | 1 oz: Blue Curacao Liqueur | 1 oz: Banana Liqueur | 2 oz: Passion Fruit Juice | 2 oz: Apple Juice | 1 tsp: Egg,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
2,Apple Fairy,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",For this recipe. make some Juice - Apple Juice ice cubes before you start. Then add 3 Juice - Apple Juice cubes to a cocktail glass. Add absinthe and apple Vodka. and fill with cider.,| 2/3 oz: Herbal Liqueur | 2/3 oz: Apple Vodka | 3 oz: Apple Soda,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
3,Black Rose Bacardi,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Shake or stir. pour it into a coctail glass. add some crushed ice and serve.,|: Ice | 1 oz: White Rum | 2 oz: Cola | 2 oz: Lime Juice,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
4,Blue Latvian,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Pour the Jack Daniel`s whiskey into a mixing glass. Add blue raspberry juice until the mixture is more blue than brown. Finish with lemon-lime soda until the mixture is fizzing. Serve in a collins glass.,| 1 oz: Bourbon Whiskey |: Raspberry Juice |: Lemon-Lime Soda,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!


In [7]:
df_ctl_info = df_ctl_info[df_ctl_info.Recipe != "Woops... We couldn't retrieve the exact recipe... It's trial & error time! Just a little more fun before enjoying a nice drink!"]

In [8]:
len(df_ctl_info)

5102

In [9]:
df_ctl_info.drop_duplicates('Name', inplace=True)

In [10]:
len(df_ctl_info)

4677

In [11]:
df_ctl_info

,Name,Description,Recipe,Ingredients,Nutrition Facts,Video Link
1,Azzuro,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Shake and strain into an ice-filled collins glass. and garnish with fruit.,| 1 oz: Exotic-Fruit Liqueur | 1 oz: Blue Curacao Liqueur | 1 oz: Banana Liqueur | 2 oz: Passion Fruit Juice | 2 oz: Apple Juice | 1 tsp: Egg,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
2,Apple Fairy,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",For this recipe. make some Juice - Apple Juice ice cubes before you start. Then add 3 Juice - Apple Juice cubes to a cocktail glass. Add absinthe and apple Vodka. and fill with cider.,| 2/3 oz: Herbal Liqueur | 2/3 oz: Apple Vodka | 3 oz: Apple Soda,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
3,Black Rose Bacardi,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Shake or stir. pour it into a coctail glass. add some crushed ice and serve.,|: Ice | 1 oz: White Rum | 2 oz: Cola | 2 oz: Lime Juice,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
4,Blue Latvian,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Pour the Jack Daniel`s whiskey into a mixing glass. Add blue raspberry juice until the mixture is more blue than brown. Finish with lemon-lime soda until the mixture is fizzing. Serve in a collins glass.,| 1 oz: Bourbon Whiskey |: Raspberry Juice |: Lemon-Lime Soda,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
5,Boston Cooler,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Stir lemon juice. sugar and 2 oz of club soda together in a collins glass. Fill the glass with cracked ice. Add rum. Top with club soda. and stir. Garnish with a spiral of orange or lemon peel. and serve.,| 2 oz: White Rum | 1 oz: Lemon Juice | 1 tsp: Superfine Sugar | 3-4 oz: Club Soda,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
...,...,...,...,...,...,...
5111,Brown Hen,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Fill glass with ice. then Vodka. then bitters. Stir.,| 2 oz: Vodka | 1 tsp tsp: Bitters,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
5112,Bosom Caresser,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",In a mixing glass half-filled with crushed ice. combine all of the ingredients. Stir well. Strain into a cocktail glass.,|: Crushed Ice | 1.5 oz: Brandy | 1 oz: Madeira | .5 oz: Triple Sec,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
5113,Brandy Water Press,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Pour the brandy over ice cubes in a

In [12]:
df_ctl_info.reset_index(inplace=True)

In [13]:
df_ctl_info.drop(columns='index', inplace=True)

In [14]:
return_info_df = df_ctl_info

In [15]:
return_info_df.to_csv('./CSVs/return_info_ctl.csv', sep=',', index=False)

### df_ctl_info ready for call-back of cocktail info after recommendation  
___________________________________________________________________  
## Fusing strings from 'Description', 'Recipe' & 'Ingredient' for vectorization and tokenizing

Create working dataframe.

In [16]:
df_ctl_dropnanrecipe = df_ctl_info

In [17]:
df_ctl_dropnanrecipe

,Name,Description,Recipe,Ingredients,Nutrition Facts,Video Link
0,Azzuro,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Shake and strain into an ice-filled collins glass. and garnish with fruit.,| 1 oz: Exotic-Fruit Liqueur | 1 oz: Blue Curacao Liqueur | 1 oz: Banana Liqueur | 2 oz: Passion Fruit Juice | 2 oz: Apple Juice | 1 tsp: Egg,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
1,Apple Fairy,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",For this recipe. make some Juice - Apple Juice ice cubes before you start. Then add 3 Juice - Apple Juice cubes to a cocktail glass. Add absinthe and apple Vodka. and fill with cider.,| 2/3 oz: Herbal Liqueur | 2/3 oz: Apple Vodka | 3 oz: Apple Soda,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
2,Black Rose Bacardi,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Shake or stir. pour it into a coctail glass. add some crushed ice and serve.,|: Ice | 1 oz: White Rum | 2 oz: Cola | 2 oz: Lime Juice,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
3,Blue Latvian,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Pour the Jack Daniel`s whiskey into a mixing glass. Add blue raspberry juice until the mixture is more blue than brown. Finish with lemon-lime soda until the mixture is fizzing. Serve in a collins glass.,| 1 oz: Bourbon Whiskey |: Raspberry Juice |: Lemon-Lime Soda,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
4,Boston Cooler,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Stir lemon juice. sugar and 2 oz of club soda together in a collins glass. Fill the glass with cracked ice. Add rum. Top with club soda. and stir. Garnish with a spiral of orange or lemon peel. and serve.,| 2 oz: White Rum | 1 oz: Lemon Juice | 1 tsp: Superfine Sugar | 3-4 oz: Club Soda,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
...,...,...,...,...,...,...
4672,Brown Hen,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Fill glass with ice. then Vodka. then bitters. Stir.,| 2 oz: Vodka | 1 tsp tsp: Bitters,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
4673,Bosom Caresser,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",In a mixing glass half-filled with crushed ice. combine all of the ingredients. Stir well. Strain into a cocktail glass.,|: Crushed Ice | 1.5 oz: Brandy | 1 oz: Madeira | .5 oz: Triple Sec,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
4674,Brandy Water Press,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Pour the brandy over ice cubes in a

Clean all the strings in concerned columns using regex

In [18]:
df_ctl_dropnanrecipe.iloc[:,1] # = 'Description'
df_ctl_dropnanrecipe.iloc[0,2] # = 'Recipe'
df_ctl_dropnanrecipe.iloc[0,3] # = 'Ingredients'

' | 1 oz: Exotic-Fruit Liqueur | 1 oz: Blue Curacao Liqueur | 1 oz: Banana Liqueur | 2 oz: Passion Fruit Juice | 2 oz: Apple Juice | 1 tsp: Egg'

In [19]:
def use_regex(input_text):
    pattern = re.sub(r'\b\d+\b|\d+\.\d+|[^\w\s]|oz|ml', '', input_text).lower()
    return pattern

In [20]:
df_ctl_dropnanrecipe.iloc[:,1] = df_ctl_dropnanrecipe.iloc[:,1].apply(lambda x: use_regex(x))

In [21]:
df_ctl_dropnanrecipe.iloc[:,2] = df_ctl_dropnanrecipe.iloc[:,2].apply(lambda x: use_regex(x))

In [22]:
df_ctl_dropnanrecipe.iloc[:,3] = df_ctl_dropnanrecipe.iloc[:,3].apply(lambda x: use_regex(x))

In [23]:
df_ctl_dropnanrecipe.head()

,Name,Description,Recipe,Ingredients,Nutrition Facts,Video Link
0,Azzuro,unfortunately we have no description for this drink youll have to describe it yourself,shake and strain into an icefilled collins glass and garnish with fruit,exoticfruit liqueur blue curacao liqueur banana liqueur passion fruit juice apple juice tsp egg,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
1,Apple Fairy,unfortunately we have no description for this drink youll have to describe it yourself,for this recipe make some juice apple juice ice cubes before you start then add juice apple juice cubes to a cocktail glass add absinthe and apple vodka and fill with cider,herbal liqueur apple vodka apple soda,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
2,Black Rose Bacardi,unfortunately we have no description for this drink youll have to describe it yourself,shake or stir pour it into a coctail glass add some crushed ice and serve,ice white rum cola lime juice,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
3,Blue Latvian,unfortunately we have no description for this drink youll have to describe it yourself,pour the jack daniels whiskey into a mixing glass add blue raspberry juice until the mixture is more blue than brown finish with lemonlime soda until the mixture is fizzing serve in a collins glass,bourbon whiskey raspberry juice lemonlime soda,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
4,Boston Cooler,unfortunately we have no description for this drink youll have to describe it yourself,stir lemon juice sugar and of club soda together in a collins glass fill the glass with cracked ice add rum top with club soda and stir garnish with a spiral of orange or lemon peel and serve,white rum lemon juice tsp superfine sugar club soda,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!


Replace "unfortunately we have no description for this drink youll have to describe it yourself" with ""  (empty string)

In [24]:
df_ctl_dropnanrecipe.iloc[:,1] = df_ctl_dropnanrecipe.iloc[:,1].replace("unfortunately we have no description for this drink youll have to describe it yourself", "")

In [25]:
df_ctl_dropnanrecipe.drop(columns=['Nutrition Facts','Video Link'], inplace=True)

In [26]:
df_ctl_dropnanrecipe.head()

,Name,Description,Recipe,Ingredients
0,Azzuro,,shake and strain into an icefilled collins glass and garnish with fruit,exoticfruit liqueur blue curacao liqueur banana liqueur passion fruit juice apple juice tsp egg
1,Apple Fairy,,for this recipe make some juice apple juice ice cubes before you start then add juice apple juice cubes to a cocktail glass add absinthe and apple vodka and fill with cider,herbal liqueur apple vodka apple soda
2,Black Rose Bacardi,,shake or stir pour it into a coctail glass add some crushed ice and serve,ice white rum cola lime juice
3,Blue Latvian,,pour the jack daniels whiskey into a mixing glass add blue raspberry juice until the mixture is more blue than brown finish with lemonlime soda until the mixture is fizzing serve in a collins glass,bourbon whiskey raspberry juice lemonlime soda
4,Boston Cooler,,stir lemon juice sugar and of club soda together in a collins glass fill the glass with cracked ice add rum top with club soda and stir garnish with a spiral of orange or lemon peel and serve,white rum lemon juice tsp superfine sugar club soda


### Fuse the string from the three last columns

In [27]:
df_ctl_dropnanrecipe['text'] = df_ctl_dropnanrecipe['Description'] + ' ' + df_ctl_dropnanrecipe['Recipe'] + ' ' + df_ctl_dropnanrecipe['Ingredients']

In [28]:
df_ctl_text = df_ctl_dropnanrecipe[['Name', 'text']]

In [29]:
df_ctl_text

,Name,text
0,Azzuro,shake and strain into an icefilled collins glass and garnish with fruit exoticfruit liqueur blue curacao liqueur banana liqueur passion fruit juice apple juice tsp egg
1,Apple Fairy,for this recipe make some juice apple juice ice cubes before you start then add juice apple juice cubes to a cocktail glass add absinthe and apple vodka and fill with cider herbal liqueur apple vodka apple soda
2,Black Rose Bacardi,shake or stir pour it into a coctail glass add some crushed ice and serve ice white rum cola lime juice
3,Blue Latvian,pour the jack daniels whiskey into a mixing glass add blue raspberry juice until the mixture is more blue than brown finish with lemonlime soda until the mixture is fizzing serve in a collins glass bourbon whiskey raspberry juice lemonlime soda
4,Boston Cooler,stir lemon juice sugar and of club soda together in a collins glass fill the glass with cracked ice add rum top with club soda and stir garnish with a spiral of orange or lemon peel and serve white rum lemon juice tsp superfine sugar club soda
...,...,...
4672,Brown Hen,fill glass with ice then vodka then bitters stir vodka tsp tsp bitters
4673,Bosom Caresser,in a mixing glass halffilled with crushed ice combine all of the ingredients stir well strain into a cocktail glass crushed ice brandy madeira triple sec
4674,Brandy Water Press,pour the brandy over ice cubes in a collins glass and fill almost to the top with water top off with the tonic water and serve brandy tonic water water
4675,Big Dipper,pour the pussers rum brandy lime juice sugar and cointreau into a cocktail shaker halffilled with ice cubes shake well and strain into an oldfashioned glass filled with ice cubes fill with club soda and serve dark rum brandy tbsp lime juice tsp sugar dashs orange liqueur club soda


In [30]:
type(df_ctl_text.iloc[0,1])

str

## Vectorize text and input into matrix

In [31]:
from nltk.stem.snowball import EnglishStemmer
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS

In [32]:
vectorizer = CountVectorizer(stop_words='english')

Extract list of string from df_ctl_text['text'] and clean

In [33]:
text = df_ctl_text['text'].tolist()

In [34]:
len(text)

4677

In [35]:
print(text)

[' shake and strain into an icefilled collins glass and garnish with fruit     exoticfruit liqueur    blue curacao liqueur    banana liqueur    passion fruit juice    apple juice   tsp egg', ' for this recipe make some juice  apple juice ice cubes before you start then add  juice  apple juice cubes to a cocktail glass add absinthe and apple vodka and fill with cider     herbal liqueur    apple vodka    apple soda', ' shake or stir pour it into a coctail glass add some crushed ice and serve   ice    white rum    cola    lime juice', ' pour the jack daniels whiskey into a mixing glass add blue raspberry juice until the mixture is more blue than brown finish with lemonlime soda until the mixture is fizzing serve in a collins glass     bourbon whiskey  raspberry juice  lemonlime soda', ' stir lemon juice sugar and   of club soda together in a collins glass fill the glass with cracked ice add rum top with club soda and stir garnish with a spiral of orange or lemon peel and serve     white r

In [36]:
clean_text = []
for x in text:
    clean_str = re.sub(r'\s+', ' ', x)
    clean_str = re.sub(r'\d+\w*', '', x)
    clean_str = re.sub(r'cl', '', x)
    clean_text.append(clean_str)

In [37]:
len(clean_text)

4677

In [38]:
clean_text

[' shake and strain into an icefilled collins glass and garnish with fruit     exoticfruit liqueur    blue curacao liqueur    banana liqueur    passion fruit juice    apple juice   tsp egg',
 ' for this recipe make some juice  apple juice ice cubes before you start then add  juice  apple juice cubes to a cocktail glass add absinthe and apple vodka and fill with cider     herbal liqueur    apple vodka    apple soda',
 ' shake or stir pour it into a coctail glass add some crushed ice and serve   ice    white rum    cola    lime juice',
 ' pour the jack daniels whiskey into a mixing glass add blue raspberry juice until the mixture is more blue than brown finish with lemonlime soda until the mixture is fizzing serve in a collins glass     bourbon whiskey  raspberry juice  lemonlime soda',
 ' stir lemon juice sugar and   of ub soda together in a collins glass fill the glass with cracked ice add rum top with ub soda and stir garnish with a spiral of orange or lemon peel and serve     white r

Vectorize

In [39]:
vectors = vectorizer.fit_transform(clean_text).todense()

In [40]:
vocabulary = vectorizer.get_feature_names_out()

In [41]:
index = df_ctl_text['Name'].tolist()

In [42]:
word_matrix = pd.DataFrame(vectors, columns=vocabulary, index=index)

In [43]:
word_matrix

,15,16,1800s,1860s,18th,1900s,1920s,1930s,1940s,1950s,1960s,1970s,1980s,1990s,19th,19thcentury,1st,1½,20,200,20min,20th,2cup,2nd,3rd,3rds,3vegas,470capacity,48rsonce,4hrs,4th,500,60s,70s,75s,7up,80s,8hourschill,90s,ab,abc,able,absente,absinthe,absinthesazerac,absolut,absolute,absolutely,absorb,absorbed,absorbs,accent,acceptable,acceptance,accepted,accessible,accompanied,accompaniment,according,accordingly,accordinglyonce,achieve,achieved,achieving,acidic,acidity,active,actual,actually,ad,add,added,addif,adding,addition,additional,additives,adds,adequate,adios,adjust,adjusted,admire,adult,adults,advance,advisable,advocaat,affect,affinity,afms,afraid,aftershock,aftertaste,againgarnish,agave,age,aged,agent,aging,agitating,agree,ahead,ailed,ailment,ails,aim,aims,airedale,airtight,al,alcohol,alcoholic,alcohols,ale,alike,alize,alizãƒâ,alleged,allegedly,alleviate,alliances,allow,allowing,allows,allsorts,allspice,allure,allures,almond,almondembed,almonds,almondstuffed,almostfill,almostfilled,aloha,alongside,alpenz,alstir,alternating,alternative,alternatively,alternatives,alters,amarelle,amaretto,amaro,amarula,amato,amazing,ambassador,amber,ambon,amd,amer,american,americano,amf,amore,amounts,amour,amp,andes,andor,anejo,angelica,angostura,anis,anise,anisette,anoth,ans,answers,antimalarial,antique,anytime,anywhe,ap,apart,aperitif,aperol,apfelkorn,apparently,appeal,appealing,appealwhat,appear,appearance,appearances,appeared,appears,apperance,apple,applecranberry,applejack,apples,applesauce,appleton,applied,apply,appreciated,appreciating,approachable,approached,appropriate,approx,approximate,approximately,apricot,aquavit,arabica,arak,archers,arent,aret,argue,arif,arizona,arkes,armagnac,aroma,aromas,aromasip,aromatic,array,arriving,arrow,articuno,artificial,artois,aside,asidein,ask,aspiring,ass,assic,assicthat,assicthe,associate,associated,association,assume,assy,astoundingly,atmosphere,atmospherethe,atop,attempt,attention,attractive,...,vermeires,vermout,vermouth,vermouths,vernors,versatile,versatility,version,versionit,versions,vertically,verycrushed,vesper,vess,vessel,veteran,vibes,vibrant,vice,video,vieux,view,vigorous,vigorously,vigourously,village,vincenzo,vinegar,viniq,violently,violet,violetbased,violette,violettes,virginians,virility,virtually,visibility,visible,visiting,visitors,visually,vitamin,vnecks,vo,voda,vodak,vodk,vodka,vodkaadd,vodkaalmond,vodkafill,vodkaonce,vodkas,vodkasoaked,vodkaspiked,vodkathe,vodkathen,vodkatop,voila,voilà,volcano,volla,volume,voyant,vs,vshape,wait,waiting,wake,wallbanger,wallop,walnut,walter,waned,want,wanted,war,warm,warmed,warmer,warming,warmpour,warmweather,warning,wars,wash,washes,waste,wasted,wastein,watch,water,watereddown,waterfloat,watergarnish,watermelon,watermeloncherry,waterpour,watershake,waterthen,way,wbatter,wcracked,wcream,weakens,weary,weather,wedge,wedges,wee,weed,week,weekends,weeks,weight,wel,welch,welchs,welcome,wellchilled,wellchilleddoublestrain,wellchilledstrain,welldont,wellgarnish,welliced,wellknown,wellmade,wellpour,wellrounded,wellstrain,wellsuited,west,wet,whalers,whats,wheel,wheels,whichever,whilst,whip,whipped,whipping,whisk,whiskey,whiskeyin,whiskeys,whiskies,whisky,white,whites,whiz,wi,wice,wide,widely,widespread,widow,wild,wildberry,wilderberry,william,windsor,wine,winemaking,winethread,winneralthough,winter,winters,wisconsin,wisely,wish,wit,withcoffee,withcream,withlemonade,withwithout,wkd,woman,wonder,wonderful,wonderfully,wondrich,wont,wooden,woodruff,wookie,worcestershire,word,work,workers,workin,works,world,worldhow,worlds,worldthe,worldwide,wormsbearsetc,wormwood,worry,worth,worthy,wouldwith,wow,wrap,wrapped,wreck,wrong,wth,wwwstoredrinklaborg,xxx,year,yearold,years,yearstraditionally,yearthe,yeast,yee,yell,yellow,yes,yi,yoda,yoghurt,yolk,yolks,york,youd,youll,young,youre,youtube,youve,yukon,yum,yummy,yvette,zambuca,zealand,zest,zesty,zigzag,zima,zinggarnish,zombie,zubrowka,zwack,ãââ,ãââãƒæãâåãƒâšãâgermeister
Azzuro,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

___________________________________________

## Train the model

In [44]:
X = word_matrix.reset_index()
X.head()

,index,15,16,1800s,1860s,18th,1900s,1920s,1930s,1940s,1950s,1960s,1970s,1980s,1990s,19th,19thcentury,1st,1½,20,200,20min,20th,2cup,2nd,3rd,3rds,3vegas,470capacity,48rsonce,4hrs,4th,500,60s,70s,75s,7up,80s,8hourschill,90s,ab,abc,able,absente,absinthe,absinthesazerac,absolut,absolute,absolutely,absorb,absorbed,absorbs,accent,acceptable,acceptance,accepted,accessible,accompanied,accompaniment,according,accordingly,accordinglyonce,achieve,achieved,achieving,acidic,acidity,active,actual,actually,ad,add,added,addif,adding,addition,additional,additives,adds,adequate,adios,adjust,adjusted,admire,adult,adults,advance,advisable,advocaat,affect,affinity,afms,afraid,aftershock,aftertaste,againgarnish,agave,age,aged,agent,aging,agitating,agree,ahead,ailed,ailment,ails,aim,aims,airedale,airtight,al,alcohol,alcoholic,alcohols,ale,alike,alize,alizãƒâ,alleged,allegedly,alleviate,alliances,allow,allowing,allows,allsorts,allspice,allure,allures,almond,almondembed,almonds,almondstuffed,almostfill,almostfilled,aloha,alongside,alpenz,alstir,alternating,alternative,alternatively,alternatives,alters,amarelle,amaretto,amaro,amarula,amato,amazing,ambassador,amber,ambon,amd,amer,american,americano,amf,amore,amounts,amour,amp,andes,andor,anejo,angelica,angostura,anis,anise,anisette,anoth,ans,answers,antimalarial,antique,anytime,anywhe,ap,apart,aperitif,aperol,apfelkorn,apparently,appeal,appealing,appealwhat,appear,appearance,appearances,appeared,appears,apperance,apple,applecranberry,applejack,apples,applesauce,appleton,applied,apply,appreciated,appreciating,approachable,approached,appropriate,approx,approximate,approximately,apricot,aquavit,arabica,arak,archers,arent,aret,argue,arif,arizona,arkes,armagnac,aroma,aromas,aromasip,aromatic,array,arriving,arrow,articuno,artificial,artois,aside,asidein,ask,aspiring,ass,assic,assicthat,assicthe,associate,associated,association,assume,assy,astoundingly,atmosphere,atmospherethe,atop,attempt,attention,...,vermeires,vermout,vermouth,vermouths,vernors,versatile,versatility,version,versionit,versions,vertically,verycrushed,vesper,vess,vessel,veteran,vibes,vibrant,vice,video,vieux,view,vigorous,vigorously,vigourously,village,vincenzo,vinegar,viniq,violently,violet,violetbased,violette,violettes,virginians,virility,virtually,visibility,visible,visiting,visitors,visually,vitamin,vnecks,vo,voda,vodak,vodk,vodka,vodkaadd,vodkaalmond,vodkafill,vodkaonce,vodkas,vodkasoaked,vodkaspiked,vodkathe,vodkathen,vodkatop,voila,voilà,volcano,volla,volume,voyant,vs,vshape,wait,waiting,wake,wallbanger,wallop,walnut,walter,waned,want,wanted,war,warm,warmed,warmer,warming,warmpour,warmweather,warning,wars,wash,washes,waste,wasted,wastein,watch,water,watereddown,waterfloat,watergarnish,watermelon,watermeloncherry,waterpour,watershake,waterthen,way,wbatter,wcracked,wcream,weakens,weary,weather,wedge,wedges,wee,weed,week,weekends,weeks,weight,wel,welch,welchs,welcome,wellchilled,wellchilleddoublestrain,wellchilledstrain,welldont,wellgarnish,welliced,wellknown,wellmade,wellpour,wellrounded,wellstrain,wellsuited,west,wet,whalers,whats,wheel,wheels,whichever,whilst,whip,whipped,whipping,whisk,whiskey,whiskeyin,whiskeys,whiskies,whisky,white,whites,whiz,wi,wice,wide,widely,widespread,widow,wild,wildberry,wilderberry,william,windsor,wine,winemaking,winethread,winneralthough,winter,winters,wisconsin,wisely,wish,wit,withcoffee,withcream,withlemonade,withwithout,wkd,woman,wonder,wonderful,wonderfully,wondrich,wont,wooden,woodruff,wookie,worcestershire,word,work,workers,workin,works,world,worldhow,worlds,worldthe,worldwide,wormsbearsetc,wormwood,worry,worth,worthy,wouldwith,wow,wrap,wrapped,wreck,wrong,wth,wwwstoredrinklaborg,xxx,year,yearold,years,yearstraditionally,yearthe,yeast,yee,yell,yellow,yes,yi,yoda,yoghurt,yolk,yolks,york,youd,youll,young,youre,youtube,youve,yukon,yum,yummy,yvette,zambuca,zealand,zest,zesty,zigzag,zima,zinggarnish,zombie,zubrowka,zwack,ãââ,ãââãƒæãâåãƒâšãâgermeister
0,Azzuro,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [45]:
X.drop(columns='index', inplace=True)

In [46]:
len(X)

4677

In [47]:
y = word_matrix.reset_index()
y.rename(columns={'index':'name'}, inplace=True)

In [48]:
y = y['name']
y

0                   Azzuro
1              Apple Fairy
2       Black Rose Bacardi
3             Blue Latvian
4            Boston Cooler
               ...        
4672             Brown Hen
4673        Bosom Caresser
4674    Brandy Water Press
4675            Big Dipper
4676     Blue Ice Mountain
Name: name, Length: 4677, dtype: object

## Split train/test

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [50]:
len(X_train)

3741

## Train model

In [51]:
from sklearn.neighbors import KNeighborsClassifier

In [52]:
model = KNeighborsClassifier(n_neighbors=10)

In [53]:
model.fit(X_train.values, y_train.values)

KNeighborsClassifier(n_neighbors=10)

_______________________________

## User input preprocessing

In [54]:
return_info_df #why is the return_info_df modified since it's last definition ??

,Name,Description,Recipe,Ingredients,text
0,Azzuro,,shake and strain into an icefilled collins glass and garnish with fruit,exoticfruit liqueur blue curacao liqueur banana liqueur passion fruit juice apple juice tsp egg,shake and strain into an icefilled collins glass and garnish with fruit exoticfruit liqueur blue curacao liqueur banana liqueur passion fruit juice apple juice tsp egg
1,Apple Fairy,,for this recipe make some juice apple juice ice cubes before you start then add juice apple juice cubes to a cocktail glass add absinthe and apple vodka and fill with cider,herbal liqueur apple vodka apple soda,for this recipe make some juice apple juice ice cubes before you start then add juice apple juice cubes to a cocktail glass add absinthe and apple vodka and fill with cider herbal liqueur apple vodka apple soda
2,Black Rose Bacardi,,shake or stir pour it into a coctail glass add some crushed ice and serve,ice white rum cola lime juice,shake or stir pour it into a coctail glass add some crushed ice and serve ice white rum cola lime juice
3,Blue Latvian,,pour the jack daniels whiskey into a mixing glass add blue raspberry juice until the mixture is more blue than brown finish with lemonlime soda until the mixture is fizzing serve in a collins glass,bourbon whiskey raspberry juice lemonlime soda,pour the jack daniels whiskey into a mixing glass add blue raspberry juice until the mixture is more blue than brown finish with lemonlime soda until the mixture is fizzing serve in a collins glass bourbon whiskey raspberry juice lemonlime soda
4,Boston Cooler,,stir lemon juice sugar and of club soda together in a collins glass fill the glass with cracked ice add rum top with club soda and stir garnish with a spiral of orange or lemon peel and serve,white rum lemon juice tsp superfine sugar club soda,stir lemon juice sugar and of club soda together in a collins glass fill the glass with cracked ice add rum top with club soda and stir garnish with a spiral of orange or lemon peel and serve white rum lemon juice tsp superfine sugar club soda
...,...,...,...,...,...
4672,Brown Hen,,fill glass with ice then vodka then bitters stir,vodka tsp tsp bitters,fill glass with ice then vodka then bitters stir vodka tsp tsp bitters
4673,Bosom Caresser,,in a mixing glass halffilled with crushed ice combine all of the ingredients stir well strain into a cocktail glass,crushed ice brandy madeira triple sec,in a mixing glass halffilled with crushed ice combine all of the ingredients stir well strain into a cocktail glass crushed ice brandy madeira triple sec
4674,Brandy Water Press,,pour the brandy over ice cubes in a collins glass and fill almost to the top with water top off with the tonic water and serve,brandy tonic water water,pour the brandy over ice cubes in a collins glass and fill almost to the top with water top off with the tonic water and serve brandy tonic water water
4675,Big Dipper,,pour the pussers rum brandy lime juice sugar and cointreau into a cocktail shaker halffilled with ice cubes shake well and strain into an oldfashioned glass filled with ice cubes fill with club soda and serve,dark rum brandy tbsp lime juice tsp sugar dashs orange liqueur club soda,pour the pussers rum brandy lime juice sugar and cointreau into a cocktail shaker halffilled with ice cubes shake well and strain into an oldfashioned glass filled with ice cubes fill with club soda and serve dark rum brandy tbsp lime juice tsp sugar dashs orange liqueur club soda


In [55]:
def preprocess_input(input):
    input = input.lower()
    input = re.sub("[0-9]", "",input)
    input = [input]
    input = vectorizer.transform(input)
    return input

summer fresh juice rum

In [56]:
user_input = input('Please enter an ingredient')
print(user_input)
input_vector = preprocess_input(user_input)
print(input_vector)

martini pina colada
  (0, 799)	1
  (0, 2556)	1
  (0, 3105)	1


In [57]:
input_vector

<1x4773 sparse matrix of type '<class 'numpy.int64'>'
	with 3 stored elements in Compressed Sparse Row format>

## Finding similar ingredients

In [58]:
return_info_df

,Name,Description,Recipe,Ingredients,text
0,Azzuro,,shake and strain into an icefilled collins glass and garnish with fruit,exoticfruit liqueur blue curacao liqueur banana liqueur passion fruit juice apple juice tsp egg,shake and strain into an icefilled collins glass and garnish with fruit exoticfruit liqueur blue curacao liqueur banana liqueur passion fruit juice apple juice tsp egg
1,Apple Fairy,,for this recipe make some juice apple juice ice cubes before you start then add juice apple juice cubes to a cocktail glass add absinthe and apple vodka and fill with cider,herbal liqueur apple vodka apple soda,for this recipe make some juice apple juice ice cubes before you start then add juice apple juice cubes to a cocktail glass add absinthe and apple vodka and fill with cider herbal liqueur apple vodka apple soda
2,Black Rose Bacardi,,shake or stir pour it into a coctail glass add some crushed ice and serve,ice white rum cola lime juice,shake or stir pour it into a coctail glass add some crushed ice and serve ice white rum cola lime juice
3,Blue Latvian,,pour the jack daniels whiskey into a mixing glass add blue raspberry juice until the mixture is more blue than brown finish with lemonlime soda until the mixture is fizzing serve in a collins glass,bourbon whiskey raspberry juice lemonlime soda,pour the jack daniels whiskey into a mixing glass add blue raspberry juice until the mixture is more blue than brown finish with lemonlime soda until the mixture is fizzing serve in a collins glass bourbon whiskey raspberry juice lemonlime soda
4,Boston Cooler,,stir lemon juice sugar and of club soda together in a collins glass fill the glass with cracked ice add rum top with club soda and stir garnish with a spiral of orange or lemon peel and serve,white rum lemon juice tsp superfine sugar club soda,stir lemon juice sugar and of club soda together in a collins glass fill the glass with cracked ice add rum top with club soda and stir garnish with a spiral of orange or lemon peel and serve white rum lemon juice tsp superfine sugar club soda
...,...,...,...,...,...
4672,Brown Hen,,fill glass with ice then vodka then bitters stir,vodka tsp tsp bitters,fill glass with ice then vodka then bitters stir vodka tsp tsp bitters
4673,Bosom Caresser,,in a mixing glass halffilled with crushed ice combine all of the ingredients stir well strain into a cocktail glass,crushed ice brandy madeira triple sec,in a mixing glass halffilled with crushed ice combine all of the ingredients stir well strain into a cocktail glass crushed ice brandy madeira triple sec
4674,Brandy Water Press,,pour the brandy over ice cubes in a collins glass and fill almost to the top with water top off with the tonic water and serve,brandy tonic water water,pour the brandy over ice cubes in a collins glass and fill almost to the top with water top off with the tonic water and serve brandy tonic water water
4675,Big Dipper,,pour the pussers rum brandy lime juice sugar and cointreau into a cocktail shaker halffilled with ice cubes shake well and strain into an oldfashioned glass filled with ice cubes fill with club soda and serve,dark rum brandy tbsp lime juice tsp sugar dashs orange liqueur club soda,pour the pussers rum brandy lime juice sugar and cointreau into a cocktail shaker halffilled with ice cubes shake well and strain into an oldfashioned glass filled with ice cubes fill with club soda and serve dark rum brandy tbsp lime juice tsp sugar dashs orange liqueur club soda


In [59]:
distances, indices = model.kneighbors(input_vector, n_neighbors=10)

In [60]:
similar_cocktails = y_train.iloc[indices[0]].values

In [61]:
similar_cocktails

array(['Chambord Sour', 'Alize Martini', 'Dr. Kevorkian', 'Evil Slider',
       'Mega Mixer', 'Blue Dog', 'Gin Dog', 'Cramit', 'London Fog 2',
       'Champerno'], dtype=object)

In [62]:
recommended_ing = []
print("Recommended cocktails:")
for ingredient in similar_cocktails:
    print(ingredient)

Recommended cocktails:
Chambord Sour
Alize Martini
Dr. Kevorkian
Evil Slider
Mega Mixer
Blue Dog
Gin Dog
Cramit
London Fog 2
Champerno


## Full cocktail info

In [63]:
results_df = return_info_df.loc[return_info_df['Name'].isin(similar_cocktails)]
results_df

,Name,Description,Recipe,Ingredients,text
739,Champerno,,pour into a wine glass,champagne licorice liqueur,pour into a wine glass champagne licorice liqueur
1066,Cramit,,fill and stir they love the name,almond liqueur cranberry juice,fill and stir they love the name almond liqueur cranberry juice
1607,Chambord Sour,,blend with ice,raspberry liqueur sours,blend with ice raspberry liqueur sours
1704,Dr. Kevorkian,,stir well and enjoy,vodka fill with pepper soda,stir well and enjoy vodka fill with pepper soda
1888,Evil Slider,,mix well and serve,vodka iced tea,mix well and serve vodka iced tea
2304,Gin Dog,,pour over ice in a pint glass and serve,gin,pour over ice in a pint glass and serve gin
2854,London Fog 2,,frappe in a highball glass,gin herbal liqueur,frappe in a highball glass gin herbal liqueur
3171,Mega Mixer,,stir everything together,vodka cranberry juice lemonade,stir everything together vodka cranberry juice lemonade
4310,Alize Martini,,mix in a martini glass,passion fruit liqueur vodka,mix in a martini glass passion fruit liqueur vodka
4433,Blue Dog,,combine both while cold,vodka grapefruit juice,combine both while cold vodka grapefruit juice


______________________________

## Testing the model & fine-tuning

In [64]:
parameters_KNN = {
    'leaf_size': (1,2,5,10,15),
    'radius': (00.1,0.05,0.1,0.2)
    }

In [65]:
search_cv = GridSearchCV(model,
                         param_grid= parameters_KNN,
                         cv=2,
                         scoring='accuracy')

In [66]:
search_cv.fit(X_train, y_train)

ValueError: n_splits=2 cannot be greater than the number of members in each class.

In [ ]:
search_cv.best_params_

search 1 = {'leaf_size': 10, 'radius': 0.3}  
search 2 = {'leaf_size': 2, 'radius': 0.1}  
search 3 = {'leaf_size': 1, 'radius': 0.1}

In [ ]:
search_cv.best_score_

__________________________________

## Final model

In [67]:
final_model = NearestNeighbors(n_neighbors=5, algorithm='auto' , leaf_size= 1 , radius= 0.1)

In [68]:
final_model.fit(X.values,y.values)

NearestNeighbors(leaf_size=1, radius=0.1)

fresh summer drink white wine

In [69]:
results_df = pd.read_csv('./CSVs/return_info_ctl.csv', sep=',')
results_df

,Name,Description,Recipe,Ingredients,Nutrition Facts,Video Link
0,Azzuro,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Shake and strain into an ice-filled collins glass. and garnish with fruit.,| 1 oz: Exotic-Fruit Liqueur | 1 oz: Blue Curacao Liqueur | 1 oz: Banana Liqueur | 2 oz: Passion Fruit Juice | 2 oz: Apple Juice | 1 tsp: Egg,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
1,Apple Fairy,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",For this recipe. make some Juice - Apple Juice ice cubes before you start. Then add 3 Juice - Apple Juice cubes to a cocktail glass. Add absinthe and apple Vodka. and fill with cider.,| 2/3 oz: Herbal Liqueur | 2/3 oz: Apple Vodka | 3 oz: Apple Soda,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
2,Black Rose Bacardi,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Shake or stir. pour it into a coctail glass. add some crushed ice and serve.,|: Ice | 1 oz: White Rum | 2 oz: Cola | 2 oz: Lime Juice,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
3,Blue Latvian,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Pour the Jack Daniel`s whiskey into a mixing glass. Add blue raspberry juice until the mixture is more blue than brown. Finish with lemon-lime soda until the mixture is fizzing. Serve in a collins glass.,| 1 oz: Bourbon Whiskey |: Raspberry Juice |: Lemon-Lime Soda,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
4,Boston Cooler,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Stir lemon juice. sugar and 2 oz of club soda together in a collins glass. Fill the glass with cracked ice. Add rum. Top with club soda. and stir. Garnish with a spiral of orange or lemon peel. and serve.,| 2 oz: White Rum | 1 oz: Lemon Juice | 1 tsp: Superfine Sugar | 3-4 oz: Club Soda,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
...,...,...,...,...,...,...
4672,Brown Hen,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Fill glass with ice. then Vodka. then bitters. Stir.,| 2 oz: Vodka | 1 tsp tsp: Bitters,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
4673,Bosom Caresser,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",In a mixing glass half-filled with crushed ice. combine all of the ingredients. Stir well. Strain into a cocktail glass.,|: Crushed Ice | 1.5 oz: Brandy | 1 oz: Madeira | .5 oz: Triple Sec,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
4674,Brandy Water Press,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Pour the brandy over ice cubes in a

In [71]:
# One cell algorithm

user_input = input('Please enter an ingredient')
print(user_input)
input_vector = preprocess_input(user_input)

distances, indices = final_model.kneighbors(input_vector)

similar_cocktails = y.iloc[indices[0]].values

recommended_ing = []
print("Recommended cocktails:")

results_df = pd.read_csv('./CSVs/return_info_ctl.csv', sep=',')

results_df = results_df.loc[results_df['Name'].isin(similar_cocktails)]
results_df

martini pina colada
Recommended cocktails:


,Name,Description,Recipe,Ingredients,Nutrition Facts,Video Link
1607,Chambord Sour,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Blend with ice.,| 1 1/4 oz: Raspberry Liqueur |: Sours,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
1704,Dr. Kevorkian,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Stir well and enjoy!,| 2 oz: Vodka | fill with: Pepper Soda,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
1888,Evil Slider,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Mix well and serve.,| 3 oz: Vodka | 5 oz: Iced Tea,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
4310,Alize Martini,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Mix in a martini glass.,| 2.5 oz: Passion Fruit Liqueur | 1 oz: Vodka,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
4433,Blue Dog,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Combine both while cold.,| 2 oz: Vodka | 5 oz: Grapefruit Juice,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!


__________________________________________

### Remarks:

* Why is the results_df' content stripped ?? make y = cocktail_id / index ?  
* why does the recommender return cocktails with other ingredients than the one entered (rum, and returns NO cocktails with rum)
* On 4000+ cocktails, some turn up quite a lot...

Try:  
* Load from external csv file. (WORKED!)
* split text on characters ?
* Check data gaps. Dummies, One Hot ?

___________________________________